In [67]:
import time
import openai
from sentence_transformers import SentenceTransformer
import torch
from dotenv import load_dotenv
import os
import pandas as pd
import pandas.io.formats.style
from transformers import AutoModel, AutoTokenizer
import chromadb
import tiktoken
from transformers import AutoTokenizer
from bs4 import BeautifulSoup
import requests
import re
from langchain.output_parsers import PydanticOutputParser
from pydantic import BaseModel
from typing import List
from langchain.prompts import ChatPromptTemplate
# from langchain.chat_models import ChatOpenAI
from langchain_openai import ChatOpenAI


### 1. Testing Embeddings

In [2]:
text = "Large language models are transforming natural language processing."

models = {
    "OpenAI ada-002": {
        "type": "openai",
        "model": "text-embedding-ada-002"
    },
    "MiniLM": {
        "type": "sbert",
        "model": "all-MiniLM-L6-v2"
    },
    "BGE Base": {
        "type": "sbert",
        "model": "BAAI/bge-base-en-v1.5"
    },
    # "E5 Base": {
    #     "type": "sbert",
    #     "model": "intfloat/e5-base"
    # },
    # "E5 Large": {
    #     "type": "sbert",
    #     "model": "intfloat/e5-large"
    # },
    "Snowflake": {
        "type": "sbert",
        "model": "Snowflake/snowflake-arctic-embed-l-v2.0"
    },
    # "Jina": {
    #     "type": "other",
    #     "model": "jinaai/jina-embeddings-v3"
    # },
    # "Solon": {
    #     "type": "solon",
    # }
}


In [29]:
jinaai/jina-embeddings-v3

NameError: name 'jinaai' is not defined

In [3]:
load_dotenv()

True

In [4]:
OPEN_AI_API_KEY = os.getenv("OPENAI_API_KEY")


In [5]:
openai.api_key = OPEN_AI_API_KEY


In [7]:
sbert_models = {}
for name, config in models.items():
    if config["type"] == "sbert":
        sbert_models[name] = SentenceTransformer(config["model"])

results = []
for name, config in models.items():
    print(f"🔍 Testowanie modelu: {name}")
    start_time = time.time()
    
    if config["type"] == "openai":
        response = openai.embeddings.create(
            input=[text],
            model=config["model"]
        )
        embedding = response.data[0].embedding
    elif config["type"] == "sbert":
        model = sbert_models[name]
        if "e5" in config["model"]:
            text_to_encode = f"query: {text}"
        else:
            text_to_encode = text
        embedding = model.encode(text_to_encode, convert_to_numpy=True)
    elif config["type"] == "other":
        model = SentenceTransformer("jinaai/jina-embeddings-v3", trust_remote_code=True)
        embedding = model.encode(text_to_encode, convert_to_numpy=True)
    else:
        tokenizer = AutoTokenizer.from_pretrained("OrdalieTech/Solon-embeddings-large-0.1")
        model = AutoModel.from_pretrained("OrdalieTech/Solon-embeddings-large-0.1")
        embedding = model.encode(text_to_encode, convert_to_numpy=True)


    
    end_time = time.time()
    elapsed = end_time - start_time
    dim = len(embedding)
    
    results.append({
        "Model": name,
        "Dimension": dim,
        "Time (s)": round(elapsed, 4)
    })


df = pd.DataFrame(results)
df.sort_values("Time (s)", inplace=True)
df.reset_index(drop=True, inplace=True)
df.style.bar(subset=["Time (s)"], color="#5fba7d").set_caption("⏱️ Porównanie embeddingów")

🔍 Testowanie modelu: OpenAI ada-002
🔍 Testowanie modelu: MiniLM
🔍 Testowanie modelu: BGE Base
🔍 Testowanie modelu: Snowflake


,Model,Dimension,Time (s)
0,MiniLM,384,0.020000
1,BGE Base,768,0.099000
2,Snowflake,1024,0.244800
3,OpenAI ada-002,1536,0.620400


### 2. Creating Vector Database

In [17]:
PATH_CHROMA = "../../chroma_db"

In [18]:
client = chromadb.PersistentClient(path=PATH_CHROMA)
collection = client.get_or_create_collection(name="interview_data")


In [19]:
embedding_model = "Snowflake/snowflake-arctic-embed-l-v2.0"

In [125]:
sentence_model = SentenceTransformer(embedding_model)

In [29]:
tokenizer = AutoTokenizer.from_pretrained(embedding_model)
tokens = tokenizer("Twój tekst", return_tensors="pt")
tokens

{'input_ids': tensor([[     0, 151477,   8484,      2]]), 'attention_mask': tensor([[1, 1, 1, 1]])}

In [41]:

url = "https://skphd.medium.com/large-language-model-llm-interview-questions-ded6264547f1"
response = requests.get(url)

In [42]:
soup = BeautifulSoup(response.text, "html.parser")
pattern = re.compile(r"^\d+\.\s?.+")

questions = []
for strong in soup.find_all("strong"):
    text = strong.get_text(strip=True)
    if pattern.match(text):
        questions.append(text)

In [155]:
url_python = "https://www.datainterview.com/blog/top-100-python-interview-questions"
response_python = requests.get(url_python)

In [43]:
print(questions[0])

2. Do you need a vector store for all text-based LLM use cases?


In [164]:
def extract_questions(h3_class):
    h3 = soup.find('h3', class_=h3_class)
    print(h3)
    if h3:
        next_ol = h3.find_next_sibling(['ol', 'ul'])  # szukamy tylko list
        if next_ol:
            return [li.get_text(strip=True) for li in next_ol.find_all('li')]
    return []

In [165]:
basic_questions = extract_questions('basic-python-interview-questions')
intermediate_questions = extract_questions('intermediate-python-interview-questions')

None
None


In [166]:
python_questions = [
    "How do you remove duplicates from a list while maintaining order?",
    "What is the purpose of the enumerate() function, and how do you use it?",
    "Explain how list slicing works, including the use of negative indices.",
    "How do you flatten a nested list in Python?",
    "What is the difference between append() and extend() in lists?",
    "How do you use the zip() function in Python?",
    "How do you find the frequency of elements in a list?",
    "What are Python comprehensions, and how do they improve code readability?",
    "Explain the difference between isinstance() and type checking with type().",
    "How do you merge multiple dictionaries in Python 3.9+?",
    "What are Python closures, and how do they work?",
    "How do you reverse the order of words in a string?",
    "What is the purpose of collections.defaultdict()?",
    "How do you check if all elements in a list are unique?",
    "What is the purpose of collections.OrderedDict()?",
    "Explain how to create a recursive function in Python.",
    "What are lambda functions, and when are they most useful?",
    "How do you create a simple iterator class in Python?",
    "What is the itertools.product() function, and where is it useful?",
    "How do you handle large files in Python without loading the entire file into memory?",
    "How do you implement a binary search algorithm in Python?",
    "Explain the concept and use of decorators in Python.",
    "How do you implement a custom sorting function using sorted() with a lambda?",
    "What is the functools.reduce() function, and how does it work?",
    "How do you implement memoization in Python?",
    "How do you use the collections.Counter() for counting elements in an iterable?",
    "Implement a function that generates all permutations of a given string.",
    "What is the use of heapq, and how do you create a min-heap?",
    "Explain the with statement and how it manages resources.",
    "How do you implement a queue using Python's deque?",
    "What is multithreading, and how does Python handle it with the GIL?",
    "Explain how to build a context manager using contextlib.",
    "How do you work with JSON data in Python?",
    "Write a function that merges overlapping intervals in a list of tuples.",
    "How do you parallelize code execution using Python's multiprocessing module?",
    "How do you handle CSV data with large file sizes using Pandas efficiently?",
    "Explain the concept of method chaining in Python.",
    "How do you use the pathlib module for file system operations?",
    "What is asyncio, and how do you use it for asynchronous programming?",
    "How do you use pandas.groupby() to perform complex data aggregation?"
]


In [117]:
class LabeledAnswer(BaseModel):
    explanation: str

class Answers(BaseModel):
    answers: List[LabeledAnswer]
    difficulty: str


parser = PydanticOutputParser(pydantic_object=Answers)

format_instructions = parser.get_format_instructions().replace("{", "{{").replace("}", "}}")

prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant that answers questions with explanations and difficulty ratings."),
    ("human", f"""Please answer the following question by generating **10 different correct answers**. 3 of them should be in 1-2 sentences, 4 of them should be in 3-4, the other 3 should be in 4-5 sentences.
For each answer, provide:
- explanation
for question provide:
- difficulty level (Easy, Medium, Hard, Very Hard)

Return the result strictly in this JSON format:
{format_instructions}

Question: {{question}}""")
])


In [118]:
llm = ChatOpenAI(model="gpt-4o", temperature=0.7, api_key=OPEN_AI_API_KEY) 

chain = prompt | llm | parser

In [119]:
result = chain.invoke({
    "question": "What is overfitting in machine learning?"
})

In [120]:
print(result.answers)

[LabeledAnswer(explanation='Overfitting occurs when a machine learning model learns the training data too well, capturing noise and outliers as if they were true patterns.'), LabeledAnswer(explanation="Overfitting happens when a model's complexity is too high relative to the amount of data available, leading it to perform poorly on unseen data."), LabeledAnswer(explanation='Overfitting is a modeling error that occurs when a model is too complex and fits the training data too closely.'), LabeledAnswer(explanation='In machine learning, overfitting is a condition where a model performs well on training data but poorly on new, unseen data. This often arises when the model captures noise and random fluctuations in the training set instead of general patterns. To prevent overfitting, techniques such as cross-validation, regularization, and simplifying the model are often employed.'), LabeledAnswer(explanation='Overfitting is a common pitfall in machine learning where a model becomes too tail

In [121]:
answers_list = []

for i, question in enumerate(questions):
    result = chain.invoke({
        "question": question
    })

    answers_list.append(result)

    print(i, end="")

01234567891011121314151617181920212223242526272829303132333435363738394041424344454647

In [124]:
answers_list[4].answers[4].explanation

"When fine-tuning LLMs, ensuring that the model retains knowledge from previously learned tasks is crucial. Catastrophic forgetting occurs when the model's neural network overwrites old information with new information, leading to a loss of previously acquired skills or knowledge. This can be mitigated by techniques such as rehearsal, regularization, or using a multi-task learning framework to maintain a balance between old and new data."

In [131]:
class MetaDatas(BaseModel):
    difficulty: str
    type_question: str

In [ ]:
    collection.add(
        ids=[doc_id],
        embeddings=[embedding],
        metadatas=[{"source": "stackoverflow", "question": doc["question"], "url": doc["url"], "version": doc["version"]}],
        documents=[chunk]
    )

In [168]:
def add_to_chromadb(ID, type_question, answers_list):
    en_id = 0
    
    for answer_list in answers_list:
        difficulty = answer_list.difficulty
        metadatas = MetaDatas(difficulty=difficulty, type_question=type_question)
        
        for idx, answer in enumerate(answer_list.answers):
            explanation = answer.explanation
            exp_id = f"{ID}{en_id}_{idx}"        
            embedding = sentence_model.encode(explanation, convert_to_numpy=True)
            
            
            collection.add(
                ids=[exp_id],
                embeddings=[embedding],
                metadatas=[{"difficulty": metadatas.difficulty, "type_question": metadatas.type_question}],
                documents=[explanation]
            )
    
        en_id += 1
        print(en_id, end="")

In [153]:
add_to_chromadb("LLM_QUESTION_", "llm", answers_list)

123456789101112131415161718192021222324252627282930313233343536373839404142434445464748

In [167]:
answers_python_list = []

for i, question in enumerate(python_questions):
    result = chain.invoke({
        "question": question
    })

    answers_python_list.append(result)

    print(i, end="")

0123456789101112131415161718192021222324252627282930313233343536373839

In [170]:
add_to_chromadb("PYTHON_QUESTION_", "python", answers_python_list)

12345678910111213141516171819202122232425262728293031323334353637383940

In [171]:
print(answers_python_list[4])

answers=[LabeledAnswer(explanation='The append() method adds a single element to the end of the list, while extend() adds all elements of an iterable to the list.'), LabeledAnswer(explanation='extend() will iterate over its argument adding each element to the list, while append() treats its argument as a single object and adds it to the list.'), LabeledAnswer(explanation='The append() function modifies the list by adding its argument as one element, whereas extend() takes an iterable and adds each of its elements to the list.'), LabeledAnswer(explanation='When you use append(), you are adding exactly one new element to the list, which could be any object, including another list. In contrast, extend() requires an iterable and adds each of its elements to the existing list. This means that appending a list will result in a list within a list, while extending with a list will combine the elements.'), LabeledAnswer(explanation='append() is typically used when you want to add a single item 

In [179]:
results = collection.query(
    query_embeddings=[sentence_model.encode("What is append in python?", convert_to_numpy=True)],
    n_results=3
)

In [180]:
print(results)

{'ids': [['PYTHON_QUESTION_4_5', 'PYTHON_QUESTION_4_2', 'PYTHON_QUESTION_4_0']], 'embeddings': None, 'documents': [['append() is a method that adds its argument as a single element at the end of a list, which can be any Python object. On the other hand, extend() takes an iterable and appends each of its elements to the list individually. This means using append() with a list will result in a nested list, while extend() will merge the elements of the iterable into the original list.', 'The append() function modifies the list by adding its argument as one element, whereas extend() takes an iterable and adds each of its elements to the list.', 'The append() method adds a single element to the end of the list, while extend() adds all elements of an iterable to the list.']], 'uris': None, 'included': ['metadatas', 'documents', 'distances'], 'data': None, 'metadatas': [[{'difficulty': 'Medium', 'type_question': 'python'}, {'type_question': 'python', 'difficulty': 'Medium'}, {'type_question':